In [1]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from ultralytics import YOLO
import wandb

# dataset.py에서 YOLODataset 클래스와 관련 함수 불러오기
from dataset import YOLODataset, collate_fn, transform

In [2]:
# wandb 설정
wandb.login()  # 이미 로그인한 경우 이 줄은 생략 가능

# 프로젝트 초기화
wandb.init(project="yolov8-person-detection", config={
    "epochs": 100,
    "batch_size": 8,
    "imgsz": 640
})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lovesjns1030. Use `wandb login --relogin` to force relogin


In [3]:
# 데이터셋 및 데이터로더 생성
img_dir = os.path.join('..', 'train', 'images')  # 이미지 디렉토리 경로
labels_dir = os.path.join('..', 'train', 'labels')  # 라벨 디렉토리 경로
dataset = YOLODataset(img_dir, labels_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=wandb.config.batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [4]:
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [5]:
# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')  # YOLOv8 모델 로드

# 모델 학습
model.train(data='data.yaml', epochs=wandb.config.epochs, imgsz=wandb.config.imgsz, 
            project='../runs', name='my_yolov8_training')

# 학습 완료 후 wandb 종료
wandb.finish()

# 학습 완료 후 wandb 종료
wandb.finish()

New https://pypi.org/project/ultralytics/8.2.30 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.28  Python-3.11.7 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=../runs, name=my_yolov8_training2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sh

RuntimeError: Dataset 'data.yaml' error  
Dataset 'data.yaml' images not found , missing path 'C:\Users\dhshs\Documents\CCTV\datasets\valid\images'
Note dataset download directory is 'C:\Users\dhshs\Documents\CCTV\datasets'. You can update this in 'C:\Users\dhshs\AppData\Roaming\Ultralytics\settings.yaml'

In [ ]:
import cv2
import torch
from ultralytics import YOLO

# 모델 로드
model_path = '../runs/my_yolov8_training/weights/best.pt'
model = YOLO(model_path)

# 비디오 파일 경로
video_path = r'C:\Users\dhshs\Documents\CCTV(2)\source\cctv_video\demo.mp4'
output_path = r'C:\Users\dhshs\Documents\CCTV(2)\source\cctv_video\demo.mp4_output.mp4'

# 비디오 캡처
cap = cv2.VideoCapture(video_path)

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 모델을 사용하여 객체 검출
    results = model(frame)
    
    # 검출된 객체들을 비디오 프레임에 그리기
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy()

        for box, score, class_id in zip(boxes, scores, class_ids):
            if score > 0.5:  # 신뢰도가 0.5 이상인 객체만 표시
                x1, y1, x2, y2 = map(int, box)
                label = f"{model.names[int(class_id)]}: {score:.2f}"
                color = (0, 255, 0)  # 녹색
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 결과 프레임을 저장
    out.write(frame)

# 모든 리소스 해제
cap.release()
out.release()


0: 384x640 9 Humans, 196.5ms
Speed: 15.6ms preprocess, 196.5ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 Humans, 26.6ms
Speed: 4.3ms preprocess, 26.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 Humans, 29.6ms
Speed: 12.3ms preprocess, 29.6ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Humans, 27.9ms
Speed: 4.9ms preprocess, 27.9ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 Humans, 34.3ms
Speed: 0.0ms preprocess, 34.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 Humans, 28.3ms
Speed: 13.9ms preprocess, 28.3ms inference, 6.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 Humans, 39.7ms
Speed: 0.0ms preprocess, 39.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 Humans, 20.9ms
Speed: 0.0ms preprocess, 20.9ms inference, 1.0ms postprocess per image at shape